# Prerequisite
You need to have a working local installation of Ansys

## Sections
### I. Transmon only: render into Ansys.
1. Run finite element eigenmode analysis. <br>
1. Plot fields and display them. <br>
1. Set up EPR junction dictionary. <br>
1. Run EPR analysis on single mode. <br>
1. Get qubit freq and anharmonicity. <br>
1. Calculate EPR of substrate.  <br>
1. (Extra: Calculate surface EPR.) <br>

### II. Resonator only: render into Ansys.
1. Run finite element eigenmode analysis. <br>
1. Plot fields and display them. <br>
1. Calculate EPR of substrate.
    
### III. Transmon & resonator: render into Ansys.
1. Run finite element eigenmode analysis. <br>
1. Plot fields and display them. <br>
1. Set up EPR junction dictionary. <br>
1. Run EPR analysis on the two modes. <br>
1. Get qubit frequency and anharmonicity. <br>

In [ ]:
%load_ext autoreload
%autoreload 2

import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings
import pyEPR as epr

In [ ]:
%metal_heading Tutorial 1: Create a single transmon qubit and analyze it using pyEPR.

In [ ]:
%metal_print Part 1. Finite Element Eigenmode Analysis

### Create the design in Metal
Setup a design of a given dimension. Dimensions will be respected in the design rendering. <br>
Note that the design size extends from the origin into the first quadrant.

In [ ]:
design = designs.DesignPlanar({}, True)
design.chips.main.size['size_x'] = '2mm'
design.chips.main.size['size_y'] = '2mm'
hfss = design.renderers.hfss

gui = MetalGUI(design)

Create a single transmon with one readout resonator and move it to the center of the chip previously defined.

In [ ]:
from qiskit_metal.components.qubits.transmon_pocket import TransmonPocket

design.delete_all_components()

q1 = TransmonPocket(design, 'Q1', options =  dict(
    pad_width = '425 um', 
    pocket_height = '650um',
    connection_pads=dict(
        readout = dict(loc_W=+1,loc_H=+1, pad_width='200um')
    )))

gui.rebuild()
gui.autoscale()

### Establish a connection between Matal and Ansys
First you will need to open Ansys. <br>
You can either open Ansys manually, or by uncommenting and executing the following cell. Note that you will need to update the enironment variable name used in this cell to match that used from your version of ansys/environment. For 2020 R2 the variable name is `ANSYSEM_ROOT202`

In [ ]:
# import subprocess, os
# cmdlist = [os.path.sep.join([os.environ['ANSYSEM_ROOT202'],'reg_ansysedt.exe']), '-shortcut']
# subprocess.call(cmdlist, cwd=os.environ['ANSYSEM_ROOT202'])

You will also need a project to be open in Ansys. You can manually open a new project. Alternatively you can uncomment and execute the following cell. NOTE: if you have executed the cell above, a new project should have automatically opened, in which case you can skip this cell.

Note that if you plan on using a previously saved project, you can open it directly using the `open_ansys()` optional parameters, so you can skip this cell.

In [ ]:
# hfss.new_ansys_project()

Finally connect this jupyter notebook session with the active Ansys project. <br>
You can optionally indicate here that you intend to open and use a previously saved project. To avoid errors, the saved project will need to contain at least one design.

In [ ]:
hfss.open_ansys()
# hfss.open_ansys('C:\\project_path\\', 'Project1')  # will open a saved project before linking the jupyter session

### Prepare and run the analysis

Create and activate an eigenmode design called "TransmonQubit".

In [ ]:
hfss.add_eigenmode_design("TransmonQubit")

Render the single qubit in Metal, called Q1, to "TransmonQubit" design in Ansys.

In [ ]:
hfss.render_design(['Q1'], [])
hfss.save_screenshot()

Set the convergence parameters and junction properties in the Ansys design. <br>
Then run the analysis and plot the convergence.

In [ ]:
# Analysis properties
setup = hfss.pinfo.setup
setup.passes = 10
print(f"""
Number of eigenmodes to find             = {setup.n_modes}
Number of simulation passes              = {setup.passes}
Convergence freq max delta percent diff  = {setup.delta_f}
""")

pinfo = hfss.pinfo
pinfo.design.set_variable('Lj', '10 nH')
pinfo.design.set_variable('Cj', '0 fF')
setup.analyze()

hfss.plot_convergences()

### Plot fields and display them
Display the Ansys modeler window and plot the E-field on the chip's surface.

In [ ]:
hfss.modeler._modeler.ShowWindow()
hfss.plot_ansys_fields('main')
hfss.save_screenshot()

Delete the newly created E-field plot to prepare for the next phase.

In [ ]:
hfss.plot_ansys_delete(['Mag_E1'])

In [ ]:
%metal_print Part 2. Energy-Participation-Ratio Analysis

### Set up EPR parameters
Specify the junctions in the model; in this case there's only one, namely 'jj'.

In [ ]:
#Non-linear (Josephson) junction
pinfo = hfss.pinfo # Project info
pinfo.junctions['jj'] = {'Lj_variable': 'Lj', 'rect': 'JJ_rect_Lj_1_rect_jj', 
                             'line': 'JJ_Lj_1_rect_jj_',  'Cj_variable': 'Cj'}
pinfo.validate_junction_info() # Check that valid names of variables and objects have been supplied
pinfo.dissipative['dielectrics_bulk']    = ['main'] # Dissipative elements: specify
# Handles  microwave analysis on eigenmode solutions
eprd = epr.DistributedAnalysis(pinfo)

Find the electric and magnetic energy stored in the substrate and the system as a whole.

In [ ]:
ℰ_elec = eprd.calc_energy_electric()
ℰ_elec_substrate = eprd.calc_energy_electric(None, 'main')
ℰ_mag = eprd.calc_energy_magnetic()

print(f"""
ℰ_elec_all       = {ℰ_elec}
ℰ_elec_substrate = {ℰ_elec_substrate}
EPR of substrate = {ℰ_elec_substrate / ℰ_elec * 100 :.1f}%

ℰ_mag    = {ℰ_mag}
""")

### Run EPR analysis

Perform EPR analysis for all modes and variations.

In [ ]:
eprd.do_EPR_analysis()

# 4a. Perform Hamiltonian spectrum post-analysis, building on mw solutions using EPR
epra = epr.QuantumAnalysis(eprd.data_filename)
epra.analyze_all_variations(cos_trunc = 8, fock_trunc = 7)

# 4b. Report solved results
swp_variable = 'Lj' # suppose we swept an optimetric analysis vs. inductance Lj_alice
epra.plot_hamiltonian_results(swp_variable=swp_variable)
epra.report_results(swp_variable=swp_variable, numeric=True)

Release Ansys session

In [ ]:
hfss.disconnect_ansys()

In [ ]:
%metal_heading Tutorial 2: Create and perform EPR analysis on a CPW resonator.

In [ ]:
%metal_print Part 1. Finite Element Eigenmode Analysis

#### Update the design in Metal
Connect the transmon to a CPW. <br>
The other end of the CPW connects to an open to ground termination.

In [ ]:
from qiskit_metal.components.connectors.open_to_ground import OpenToGround
from qiskit_metal.components.interconnects.meandered import RouteMeander
otg = OpenToGround(design, 'open_to_ground', options=dict(pos_x='1.75mm',  pos_y='0um', orientation='0'))
RouteMeander(design, 'readout',  Dict(
        total_length='6 mm',
        fillet='90 um',
        lead = dict(start_straight='100um'),
        pin_inputs=Dict(
            start_pin=Dict(component='Q1', pin='readout'),
            end_pin=Dict(component='open_to_ground', pin='open')), ))

gui.rebuild()
gui.autoscale()

#### Render the resonator Design in Ansys
Connect again to Ansys. <br>
Then create and activate a separate eigenmode design called "Readout".

In [ ]:
hfss.open_ansys()
hfss.add_eigenmode_design("Readout")

Render the readout and an open to ground termination. <br>
Add open endcaps to both ends of the readout.

In [ ]:
hfss.render_design(['readout', 'open_to_ground'], [('readout', 'start'), ('readout', 'end')])
hfss.save_screenshot()

#### Prepare and run the analysis, then plot E-field

Set variables in the Ansys design.

In [ ]:
# Analysis properties
setup = hfss.pinfo.setup
setup.passes = 10
print(f"""
Number of eigenmodes to find             = {setup.n_modes}
Number of simulation passes              = {setup.passes}
Convergence freq max delta percent diff  = {setup.delta_f}
""")

pinfo = hfss.pinfo
pinfo.design.set_variable('Lj', '10 nH')
pinfo.design.set_variable('Cj', '0 fF')
setup.analyze()

hfss.plot_convergences()

Recover eigenmode frequencies for each variation.

In [ ]:
eprd = epr.DistributedAnalysis(hfss.pinfo)
eprd.get_ansys_frequencies_all()

Display the Ansys modeler window and plot the E-field on the chip's surface.

In [ ]:
hfss.modeler._modeler.ShowWindow()
hfss.plot_ansys_fields('main')
hfss.save_screenshot()

Delete the newly created E-field plot.

In [ ]:
hfss.plot_ansys_delete(['Mag_E1'])

#### Try again with a larger number of steps (better convergence)

Update the number of passes, analyze, and plot convergence data.

In [ ]:
setup.passes = 15
setup.analyze()
hfss.plot_convergences()

Display the Ansys modeler window again and plot the E-field on the chip's surface with this updated number of passes. <br>
Note that the bright areas have become much smoother compared to the previous image, indicating better convergence.

In [ ]:
hfss.modeler._modeler.ShowWindow()
hfss.plot_ansys_fields('main')
hfss.save_screenshot()

In [ ]:
%metal_print Part 2. Energy-Participation-Ratio Analysis

Find the electric and magnetic energy stored in the substrate and the system as a whole.

In [ ]:
ℰ_elec = eprd.calc_energy_electric()
ℰ_elec_substrate = eprd.calc_energy_electric(None, 'main')
ℰ_mag = eprd.calc_energy_magnetic()

print(f"""
ℰ_elec_all       = {ℰ_elec}
ℰ_elec_substrate = {ℰ_elec_substrate}
EPR of substrate = {ℰ_elec_substrate / ℰ_elec * 100 :.1f}%

ℰ_mag_all       = {ℰ_mag}
ℰ_mag % of ℰ_elec_all  = {ℰ_mag / ℰ_elec * 100 :.1f}%
""")

Release Ansys session

In [ ]:
hfss.disconnect_ansys()

In [ ]:
%metal_heading Tutorial 3: Analyze the combined transmon + CPW resonator system. 

In [ ]:
%metal_print Part 1. Finite Element Eigenmode Analysis

Connect to Ansys, then create and activate an eigenmode Ansys design called "TransmonResonator".

In [ ]:
hfss.open_ansys()
hfss.add_eigenmode_design("TransmonResonator")

Render everything in the Metal design, including both the transmon and the readout. <br>
Add an open endcap to the readout.

In [ ]:
hfss.render_design(['Q1', 'readout', 'open_to_ground'], [('readout', 'end')])
hfss.save_screenshot()

Set variables in the Ansys design. Note that this time we seek 2 eigenmodes - one with stronger fields near the transmon, the other with stronger fields near the resonator.

In [ ]:
# Analysis properties
setup = hfss.pinfo.setup
setup.n_modes = 2
setup.passes = 10
print(f"""
Number of eigenmodes to find             = {setup.n_modes}
Number of simulation passes              = {setup.passes}
Convergence freq max delta percent diff  = {setup.delta_f}
""")

pinfo = hfss.pinfo
pinfo.design.set_variable('Lj', '10 nH')
pinfo.design.set_variable('Cj', '0 fF')
setup.analyze()

hfss.plot_convergences()

Display the Ansys modeler window again and plot the E-field on the chip's surface.

In [ ]:
hfss.modeler._modeler.ShowWindow()
hfss.plot_ansys_fields('main')
hfss.save_screenshot()

Delete the newly created E-field plot.

In [ ]:
hfss.plot_ansys_delete(['Mag_E1'])

In [ ]:
%metal_print Part 2. Energy-Participation-Ratio Analysis

Specify the junctions in the model; in this case there's only one, namely 'jj'.

In [ ]:
#Non-linear (Josephson) junction
pinfo = hfss.pinfo 
pinfo.junctions['jj'] = {'Lj_variable': 'Lj',  'rect': 'JJ_rect_Lj_1_rect_jj', 
                             'line': 'JJ_Lj_1_rect_jj_',  'Cj_variable': 'Cj'}
pinfo.validate_junction_info() # Check that valid names of variables and objects have been supplied
pinfo.dissipative['dielectrics_bulk']    = ['main'] # Dissipative elements: specify
# Handles  microwave analysis on eigenmode solutions
eprd = epr.DistributedAnalysis(pinfo)

Find the electric and magnetic energy stored in the substrate and the system as a whole.

In [ ]:
ℰ_elec = eprd.calc_energy_electric()
ℰ_elec_substrate = eprd.calc_energy_electric(None, 'main')
ℰ_mag = eprd.calc_energy_magnetic()

print(f"""
ℰ_elec_all       = {ℰ_elec}
ℰ_elec_substrate = {ℰ_elec_substrate}
EPR of substrate = {ℰ_elec_substrate / ℰ_elec * 100 :.1f}%

ℰ_mag_all       = {ℰ_mag}
ℰ_mag % of ℰ_elec_all  = {ℰ_mag / ℰ_elec * 100 :.1f}%
""")

Perform EPR analysis for all modes and variations.

In [ ]:
eprd.do_EPR_analysis()

# 4a. Perform Hamiltonian spectrum post-analysis, building on mw solutions using EPR
epra = epr.QuantumAnalysis(eprd.data_filename)
epra.analyze_all_variations(cos_trunc = 8, fock_trunc = 7)

# 4b. Report solved results
swp_variable = 'Lj' # suppose we swept an optimetric analysis vs. inductance Lj_alice
epra.plot_hamiltonian_results(swp_variable=swp_variable)
epra.report_results(swp_variable=swp_variable, numeric=True)

Release Ansys's session

In [ ]:
hfss.disconnect_ansys()